In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,round

In [3]:
spark = SparkSession.builder.appName("Dataset_Description").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/26 09:38:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Competitions

In [5]:
competitions = spark.read.csv('/home/aniss/xG-model/Data/competitions.csv', header=True, inferSchema=True)

In [6]:
competitions.printSchema()

root
 |-- competition_id: integer (nullable = true)
 |-- season_id: integer (nullable = true)
 |-- country_name: string (nullable = true)
 |-- competition_name: string (nullable = true)
 |-- competition_international: boolean (nullable = true)
 |-- season_name: string (nullable = true)



In [26]:
# Number of features and rows
len(competitions.columns), competitions.count()

(6, 26)

In [8]:
competitions.show(5)

+--------------+---------+--------------------+--------------------+-------------------------+-----------+
|competition_id|season_id|        country_name|    competition_name|competition_international|season_name|
+--------------+---------+--------------------+--------------------+-------------------------+-----------+
|           223|      282|       South America|        Copa America|                     true|       2024|
|            55|      282|              Europe|           UEFA Euro|                     true|       2024|
|             9|      281|             Germany|       1. Bundesliga|                    false|  2023/2024|
|            44|      107|United States of ...| Major League Soccer|                    false|       2023|
|          1267|      107|              Africa|African Cup of Na...|                     true|       2023|
+--------------+---------+--------------------+--------------------+-------------------------+-----------+
only showing top 5 rows



In [ ]:
competitions.groupBy('country_name','competition_name').count().orderBy('country_name').show(20,False)

+------------------------+----------------------+-----+
|country_name            |competition_name      |count|
+------------------------+----------------------+-----+
|Africa                  |African Cup of Nations|1    |
|England                 |Premier League        |1    |
|Europe                  |Champions League      |5    |
|Europe                  |UEFA Euro             |2    |
|France                  |Ligue 1               |3    |
|Germany                 |1. Bundesliga         |2    |
|International           |FIFA World Cup        |2    |
|Italy                   |Serie A               |1    |
|South America           |Copa America          |1    |
|Spain                   |La Liga               |7    |
|United States of America|Major League Soccer   |1    |
+------------------------+----------------------+-----+



### Matches

In [28]:
matches = spark.read.csv('/home/aniss/xG-model/Data/matches.csv', header=True, inferSchema=True)

In [29]:
matches.printSchema()

root
 |-- match_id: integer (nullable = true)
 |-- match_date: date (nullable = true)
 |-- competition: string (nullable = true)
 |-- season: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- match_week: integer (nullable = true)
 |-- competition_stage: string (nullable = true)
 |-- stadium: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- home_managers: string (nullable = true)
 |-- away_managers: string (nullable = true)



In [30]:
# Number of features and rows
len(matches.columns), matches.count()

(14, 2450)

In [31]:
matches.show(5,truncate=False)

+--------+----------+----------------------------+------+---------+---------+----------+----------+----------+-----------------+-----------------------+-----------------------------+------------------------------+------------------------------+
|match_id|match_date|competition                 |season|home_team|away_team|home_score|away_score|match_week|competition_stage|stadium                |referee                      |home_managers                 |away_managers                 |
+--------+----------+----------------------------+------+---------+---------+----------+----------+----------+-----------------+-----------------------+-----------------------------+------------------------------+------------------------------+
|3943077 |2024-07-15|South America - Copa America|2024  |Argentina|Colombia |1         |0         |6         |Final            |Hard Rock Stadium      |Raphael Claus                |Lionel Sebastián Scaloni      |Néstor Gabriel Lorenzo        |
|3943076 |2024-07-14

In [32]:
# Number of matches per competition
matches.groupBy('competition').count().orderBy('count').show(28,False)

+----------------------------------------------+-----+
|competition                                   |count|
+----------------------------------------------+-----+
|Europe - Champions League                     |5    |
|United States of America - Major League Soccer|6    |
|South America - Copa America                  |32   |
|Africa - African Cup of Nations               |52   |
|Europe - UEFA Euro                            |102  |
|International - FIFA World Cup                |128  |
|Germany - 1. Bundesliga                       |340  |
|Italy - Serie A                               |380  |
|England - Premier League                      |380  |
|France - Ligue 1                              |435  |
|Spain - La Liga                               |590  |
+----------------------------------------------+-----+



In [33]:
matches.groupBy().avg('home_score','away_score') \
    .withColumn('home_score',round(col('avg(home_score)'),3)) \
    .withColumn('away_score',round(col('avg(away_score)'),3)) \
    .drop('avg(home_score)','avg(away_score)') \
.show()

+----------+----------+
|home_score|away_score|
+----------+----------+
|     1.539|     1.198|
+----------+----------+



### Lineups

In [34]:
lineups = spark.read.csv('/home/aniss/xG-model/Data/lineups.csv', header=True, inferSchema=True)

In [35]:
lineups.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- player_name: string (nullable = true)
 |-- player_nickname: string (nullable = true)
 |-- jersey_number: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- cards: string (nullable = true)
 |-- positions: string (nullable = true)



In [36]:
# Number of features and rows
len(lineups.columns), lineups.count()

(7, 95630)

In [37]:
lineups.show(5,truncate=False)

+---------+-------------------------------+------------------+-------------+---------+----------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|player_id|player_name                    |player_nickname   |jersey_number|country  |cards                                                                                   |positions                                    